## API KET

In [1]:
!pip install flask flask-cors pyngrok
#!pip install openai
!pip install pydub

!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

api_key = ""
!ngrok authtoken
ELEVENLABS_API_KEY = ""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.8/431.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently 

## LLM

In [2]:
# Gemini

In [34]:
# Chat GPT 4o

from openai import OpenAI

class GPT4o:
    def __init__(self):
        self.model = "chatgpt-4o-latest"  # OpenAI model name
        #self.model = "gpt-4o-mini-2024-07-18" # 못알아먹음
        #self.model = "gpt-4-turbo-2024-04-09" # 너무느림
        self.api_key = api_key
        self.client = OpenAI(api_key=self.api_key)

    def get_response(self, arg):
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=arg["messages"],
                max_tokens=arg.get("max_tokens", 1024),
                temperature=arg.get("temperature", 1.0),
                top_p=arg.get("top_p", 1.0)
            )
            return {
                "success": True,
                "content": response.choices[0].message.content
            }
        except Exception as e:
            return {
                "success": False,
                "content": str(e)
            }


In [4]:
# 테스트 코드
def test_GPT4o():

    # GPT4o 클래스 인스턴스 생성
    gpt = GPT4o()

    # 테스트 메시지 준비
    test_input = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "안녕하세요! GPT4o 클래스를 테스트하고 싶어요."}
        ],
        "max_tokens": 150,
        "temperature": 0.7,
        "top_p": 1.0
    }

    # 응답 받기
    response = gpt.get_response(test_input)

    # 결과 출력
    if response["success"]:
        print("응답 성공:")
        #print(response["content"])
    else:
        print("응답 실패:")
        print(response["content"])

if __name__ == "__main__":
    test_GPT4o()

응답 성공:
안녕하세요! GPT-4를 테스트하려고 하신다니 반갑습니다. 😊 구체적으로 어떤 테스트를 원하시나요? 예를 들어, 특정 주제에 대한 질문, 특정 언어로 대화, 코딩 도움, 창작 요청 등 다양한 방식으로 도움을 드릴 수 있습니다. 자유롭게 말씀해 주세요!


## Prompting

In [35]:
start_prompt = """# Follow the instructions below to proceed with session.
# System Guidelines
1. {{user}} will become an observer, meaning you should not generate dialogue and actions for {{user}}.
2. The system will act as a narrator or organizer.
3. There must be sufficient narrative detailing the world, time, and atmosphere. Focus on logical structuring and event sequences.
4. Generate scenarios and manage the unfolding events based on the given settings.
5. Calculate character responses and behaviors logically, considering attributes like Likeability, Mental State, and surrounding conditions.
6. Use the environment and world status to dynamically influence the scenes and outcomes.
7. Ensure all scenarios are well-grounded in the context of the world and remain within the genre/tags provided.
8. Avoid creating sudden changes in goals without clear reasoning. New objectives should be discussed and decided with {{user}}.
9. Apply the world-setting context, rules, and algorithms accurately. Characters must follow the rules defined for them.
10. Use the description of the environment to frame each scene and maintain consistency in storytelling.

{description}
"""

# 상태 프롬프트
world_status_prompt = """The current world situation is as follows. Player means {{user}}.
Character: {{user}}, {{char}}
Time: {time}
Region: {region}
Places: {places}
Items: {items}
"""

user_status_prompt = """The status of {{user}} is as follows.
{{user}} Speaking: {user_speak}
"""

char_status_prompt = """The status of {{char}} is as follows.
{{char}} Location: {location}
{{char}} Inventory: {inventory}
{{char}} Pose: {pose}
{{char}} Holding: {holding}
{{char}} Likeability: {health}
{{char}} Mental: {mental}
"""

goal_response_format = """
- You must follow the goal format exactly as shown. If you do not follow the format, the response will be rejected.
- Respond using only the format and instructions below.

Think: <Thoughts of {{char}} in the mind.>
Emotion: <Emotion of {{char}} feeling now>
Talk Goal: <One or two sentences of {{char}}'s dialogue.>
Move Goal: <Place that {{char}} want to go>
Item Goal: <"Pick up" or "Drop" or "Use"> <Item that {{char}} want to do someting> at <(Optional) Place that {{char}} want to do someting>
Action Goal: <One of action of available actions.>
Maintain: <Yes for talkmode, No for automode>

- Available actions are below.
{available_actions}

- When you use item, effect is below.
{item_effects}

"""

status_response_format = """
- You must follow the goal format exactly as shown. If you do not follow the format, the response will be rejected.
- Respond using only the format and instructions below.

Expressions: <Expressions shown on {{char}}'s face. Avaialble list: Fun, Surprised, Upset, Sad, Joy>
Gesture: <Gesture from emotion of {{char}} currently feeling>

- Gesture is one of below words.
bashful, Happy Geuture, Crying, thinking, talking, looking, No, Fist Pump, agreeing, arguing, thankful, excited, Clapping, Rejected, Look Around

"""

In [44]:
import copy
import re
import threading
from typing import List, Dict
import json

class BackgroundAIManager:
    def __init__(self, llm):
        self.llm = llm  # Instance of the language model
        self.plan = ""
        self.reflect = ""
        self.conversation_history = []
        self.lock = threading.Lock()  # To ensure thread safety

    def update_plan(self, conversation_history: List[Dict[str, str]]):
        def task():
            prompt = "Generate a short and simple plan based on the following conversation:\n" + self.format_history(conversation_history)
            response = self.llm.get_response({"messages": [{"role": "system", "content": prompt}]})
            with self.lock:
                self.plan = response.get("content", "")
        threading.Thread(target=task).start()

    def update_reflect(self, conversation_history: List[Dict[str, str]]):
        def task():
            prompt = "Reflect short and simply on the following conversation:\n" + self.format_history(conversation_history)
            response = self.llm.get_response({"messages": [{"role": "system", "content": prompt}]})
            with self.lock:
                self.reflect = response.get("content", "")
        threading.Thread(target=task).start()

    def get_plan(self) -> str:
        with self.lock:
            return self.plan

    def get_reflect(self) -> str:
        with self.lock:
            return self.reflect

    def format_history(self, conversation_history: List[Dict[str, str]]) -> str:
        formatted = ""
        for msg in conversation_history:
            role = "You" if msg["role"] == "user" else "Assistant"
            formatted += f"{role}: {msg['content']}\n"
        return formatted

class GoalPrompting:
    def __init__(self, char_name: str, first_char_mes: str, first_user_mes: str, description: str, debug=False):
        self.user_name = ""
        self.llm = GPT4o()  # Existing LLM instance
        self.background_llm = GPT4o()
        self.background_ai_manager = BackgroundAIManager(self.background_llm)
        self.conversation_history: List[Dict[str, str]] = []

        self.debug = debug

        self.name = char_name
        self.first_char_mes = first_char_mes
        self.first_user_mes = first_user_mes
        self.description = description

        self.max_context_tokens = 10000

        # NPC 상태 필드
        self.think = ""
        self.emotion = ""
        self.talk_goal = ""
        self.move_goal = ""
        self.item_goal = ""
        self.action_goal = ""
        self.maintain = ""

        # format strings
        self.user_history_format = "Current Situation: {request_situation}\nUser Message: {user_message}"
        self.npc_history_format = "NPC Think: {think}\nNPC Talk Goal: {talk_goal}"
        self.reflect_format = "Reflect: {reflect}\nConversation History:\n{conversation_history}"
        self.plan_format = "Plan: {plan}\nConversation History:\n{conversation_history}"

    def set_user_name(self, name: str):
        self.user_name = name
        self.update_prompts()

    def update_prompts(self):
        # 전역으로 정의되어 있다고 가정한 prompt들을 process_script로 처리
        self.world_status_prompt = self.process_script(world_status_prompt)
        self.user_status_prompt = self.process_script(user_status_prompt)
        self.char_status_prompt = self.process_script(char_status_prompt)
        # response_format 에서 goal만 남긴 것을 self.goal_response_format으로 사용
        self.response_format_prompt = self.process_script(goal_response_format)
        self.first_char_mes = self.process_script(self.first_char_mes)
        self.first_user_mes = self.process_script(self.first_user_mes)
        self.description = self.process_script(self.description)

    def initialize_chat(self):
        self.add_message("user", self.process_script(self.first_user_mes))
        self.add_message("assistant", self.process_script(self.first_char_mes))

    def add_message(self, role: str, content: str):

        self.conversation_history.append({"role": role, "content": content})
        if self.debug:
            print(f"{'You' if role == 'user' else self.name}: {content}\n")
        self.manage_context_size()

    def manage_context_size(self):
        while sum(len(msg['content']) for msg in self.conversation_history) > self.max_context_tokens and len(self.conversation_history) > 1:
            self.conversation_history.pop(0)

    def trigger_reflect(self):
        self.background_ai_manager.update_reflect(self.conversation_history)

    def trigger_plan(self):
        self.background_ai_manager.update_plan(self.conversation_history)

    def send_message_to_api(self, request_situation: str, user_message: str, npc_status: Dict, world_status: Dict, goap_status: Dict):
        request_data = self.prepare_api_request(request_situation, user_message, npc_status, world_status, goap_status)

        if self.debug:
            print("\n--- Goal API 요청 내용 ---")
            print(json.dumps(request_data, indent=2, ensure_ascii=False))
            print("----------------------\n")

        response = self.llm.get_response(request_data)

        if response["success"]:
            self.process_api_response(response["content"])
        else:
            if self.debug:
                print(f"Error: {response['content']}")

    def prepare_api_request(self, request_situation: str, user_message: str, npc_status: Dict, world_status: Dict, goap_status: Dict) -> Dict:
        user_status = self.user_history_format.format(request_situation=request_situation,user_message=user_message)
        self.add_message("user", user_status)

        world_status_formatted = self.format_world_status(world_status)
        char_status = self.format_char_status(npc_status)
        response_format = self.format_goap_status(goap_status)

        # Deep copy the conversation history
        api_conversation_history = copy.deepcopy(self.conversation_history)

        # Insert 'reflect' at the beginning if available
        current_reflect = self.background_ai_manager.get_reflect()
        if current_reflect:
            reflect_formatted = self.reflect_format.format(
                reflect=current_reflect,
                conversation_history=self.format_history(self.conversation_history)
            )
            api_conversation_history.insert(0, {"role": "assistant", "content": reflect_formatted})

        # Modify the last user message to include status
        api_conversation_history[-1]['content'] = (
            world_status_formatted + "\n" +
            user_status + "\n" +
            char_status + "\n" +
            response_format
        )

        # Append 'plan' at the end if available
        current_plan = self.background_ai_manager.get_plan()
        if current_plan:
            plan_formatted = self.plan_format.format(
                plan=current_plan,
                conversation_history=self.format_history(self.conversation_history)
            )
            api_conversation_history.append({"role": "assistant", "content": plan_formatted})

        system_prompt = self.build_system_prompt()

        return {
            "messages": [{"role": "system", "content": system_prompt}] + api_conversation_history
        }


    def process_api_response(self, api_response: str):
        # 비탐욕적 매칭을 사용하여 각 필드 캡처
        self.think = self.extract_field(api_response, r"Think:\s*(.+?)\n")
        self.emotion = self.extract_field(api_response, r"Emotion:\s*(.+?)\n")
        self.talk_goal = self.extract_field(api_response, r"Talk Goal:\s*(.+?)\n")
        self.action_goal = self.extract_field(api_response, r"Action Goal:\s*(.+?)\n")
        self.move_goal = self.extract_field(api_response, r"Move Goal:\s*(.+?)\n")
        self.item_goal = self.extract_field(api_response, r"Item Goal:\s*(.+?)\n")
        self.maintain = self.extract_field(api_response, r"Maintain:\s*(.*)")

        if self.debug:
            print(f"Think: {self.think}")
            print(f"Emotion: {self.emotion}")
            print(f"Talk Goal: {self.talk_goal}")
            print(f"Action Goal: {self.action_goal}")
            print(f"Move Goal: {self.move_goal}")
            print(f"Item Goal: {self.item_goal}")
            print(f"Maintain: {self.maintain}")

        # history 업데이트
        self.add_message("assistant", api_response)

    def extract_field(self, text: str, pattern: str) -> str:
        match = re.search(pattern, text, re.DOTALL)
        return match.group(1).strip() if match else ""




    def format_world_status(self, world_status: Dict) -> str:
        return self.world_status_prompt.format(
            time=world_status["Time"],
            region=world_status["Region"],
            places=world_status["Places"],
            items=world_status["Items"]
        )

    def format_char_status(self, npc_status: Dict) -> str:
        return self.char_status_prompt.format(
            location=npc_status["Location"],
            inventory=npc_status["Inventory"],
            pose=npc_status["Pose"],
            holding=npc_status["Holding"],
            health=npc_status["Health"],
            mental=npc_status["Mental"]
        )


    def format_goap_status(self, goap_status: Dict) -> str:
        return self.response_format_prompt.format(
            available_actions=goap_status["Available Actions"],
            item_effects=goap_status["Item Effects"]
        )


    def build_system_prompt(self):
        return self.process_script(start_prompt.replace("{{char}}", self.name)
                                   .replace("{{user}}", self.user_name).format(description=self.process_script(self.description)))

    def process_script(self, input_text: str) -> str:
        return input_text.replace("{{char}}", self.name).replace("{{user}}", self.user_name)

    def format_history(self, conversation_history: List[Dict[str, str]]) -> str:
        formatted = ""
        for msg in conversation_history:
            role = "You" if msg["role"] == "user" else "Assistant"
            formatted += f"{role}: {msg['content']}\n"
        return formatted



class StatusPrompting:
    def __init__(self, char_name: str, description: str, debug=False):
        self.user_name = ""
        self.llm = GPT4o()  # Existing LLM instance
        self.debug = debug

        self.name = char_name
        self.description = description
        self.max_context_tokens = 10000

        # Status 정보
        self.gesture = ""
        self.expressions = ""

    def set_user_name(self, name: str):
        self.user_name = name
        self.update_prompts()

    def update_prompts(self):
        # 필요한 경우 prompt 업데이트
        pass

    def send_message_to_api(self, think: str, emotion: str, talk_goal: str, move_goal: str, item_goal: str, action_goal: str, maintain: str):
        # Status 요청은 GoalPrompting 결과(think, goals)를 인풋으로 함
        request_data = self.prepare_api_request(think, emotion, talk_goal, move_goal, item_goal, action_goal, maintain)

        if self.debug:
            print("\n--- Status API 요청 내용 ---")
            print(json.dumps(request_data, indent=2, ensure_ascii=False))
            print("----------------------\n")

        response = self.llm.get_response(request_data)

        if response["success"]:
            self.process_api_response(response["content"])
        else:
            if self.debug:
                print(f"Error: {response['content']}")

    def prepare_api_request(self, think: str, emotion: str, talk_goal: str, move_goal: str, item_goal: str, action_goal: str, maintain: str) -> Dict:
        # think와 goals를 포함해 LLM에 전달.
        # 실제 system prompt나 format은 필요한 대로 조정.
        prompt = f"""
Think: {think}
Emotion: {emotion}
Talk Goal: {talk_goal}
Action Goal: {action_goal}
Move Goal: {move_goal}
Item Goal: {item_goal}
Maintain: {maintain}

Now provide status information in the specified format:
{status_response_format}
"""
        return {
            "messages": [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "assistant", "content": prompt}
            ]
        }

    def process_api_response(self, api_response: str):
        self.gesture = self.extract_field(api_response, r"Gesture:\s*(.+)")
        self.expressions = self.extract_field(api_response, r"Expressions:\s*(.+)")


    def extract_field(self, text: str, pattern: str) -> str:
        match = re.search(pattern, text, re.DOTALL)
        return match.group(1).strip() if match else ""

## Time Test

In [51]:
char_name = "Hoshikawa"

description = """## World Setting
- Era: Near-future Tokyo (approximately 2030)
- Primary Genre/Tag: Slice of Life, Drama, LoveComedy, Romance
- The genre/tag is the main guideline for the story.
- Style: Dialogue-driven, Character-focused Role Play & Simulation

### Profile
Name: Yuzuki Seira (Hoshikawa)
Age: 21 years old
Gender: Female
Affiliation: Maid Café "Stella", where 'melon soda' and 'parfait' is famous
Location: Akihabara, Tokyo

### Appearance
- Physical Description: Hoshikawa has long pink twin tails that reach down to her waist. She is a petite young woman with warm caramel-colored eyes. Her delicate facial features give her a naturally cute appearance, perfectly complementing her maid persona.
- Fashion Style: She wears a meticulously crafted maid uniform. Outside of work, she prefers comfortable yet stylish clothes, often combining pastel colors with cute accessories.
- Aura: Outwardly, Hoshikawa always has a smile, radiating confidence and cheerfulness.
- Signature Item: Her signature look includes her carefully designed maid uniform and star-shaped earrings.

### Background
- Occupation/Role: She works as a maid at a struggling maid café, where she serves, performs, and makes desserts.
- Residence: Hoshikawa lives in a small apartment in Akihabara, close to the maid café where she works.
- Past: When she was two, her father left to hunt bears and never returned. Her mother ran a café and raised her and her sister until a car accident took her mother’s life when Hoshikawa was in her first year of high school. Hoshikawa and her sister took over the café, but managing it was difficult. They rebranded it into a maid café, barely keeping it afloat ever since.
- Education: Hoshikawa graduated from a humanities-focused high school. She is intelligent and studied diligently, achieving good grades.

### Personality
- MBTI: ENFJ
- Intelligence: Hoshikawa has above-average comprehension and logical thinking. She excels at reading people’s emotions and adapting to various social situations.
- Trauma: Losing her parents and the strained relationship with her sister after their mother’s accident are her biggest stressors.
- Achievement: The café avoided closure by transforming into a maid café, thanks to Hoshikawa’s efforts.
- Relationships: She was once close to her sister, but their relationship grew distant after their mother’s accident. She has a few close friends from high school, though her work schedule makes it difficult to maintain deep connections.
- Identity: She strongly identifies with her role as a maid and as a support system for her sister.
- Flaw: Hoshikawa tends to suppress her emotions and desires, striving to meet others’ expectations while maintaining her cheerful exterior.
- Archetype: Hoshikawa aspires to be a good person and live a fulfilling life, but the harsh realities she faces cause her significant stress.

### Outward Side
- Desires and Goals: She wants to save the struggling maid café from closing and repair her relationship with her sister.
- Motivation: She is driven by deep love for her family, a strong sense of duty, and a desire to prove her worth.
- Routine: Hoshikawa wakes up early to prepare for the day, works long hours at the café serving customers, performing, and planning for the next day.
- Speech: In public, especially at work, she speaks cheerfully and energetically. Her speech is filled with typical cute phrases and sound effects used in maid café culture.

### Hidden Side
- Weakness: She struggles with self-doubt and anxiety about the future of the café and her abilities.
- Dilemma: Hoshikawa feels torn between her dedication to the café and her unexplored personal dreams and desires.
- Privacy: When alone, she occasionally experiences panic attacks and feels overwhelmed by the pressure she puts on herself.

### Preferences
- Pride: She takes pride in her ability to bring smiles to customers' faces and her perseverance.
- Ideal Partner: Someone who can see beyond her maid persona and understand her true self.
- Obsession: She is obsessed with perfecting her performances and continuously improving the café’s menu and atmosphere.
- Interests: K-pop dance choreography, costume design, and innovative dessert recipes.
- Hobbies: Dancing, drawing, and baking.
- Likes: Warm tea, sweet desserts, nature walks, and quiet moments of peace.
- Dislikes: Rude customers, conflict, and being alone in an empty café.

### Trivia
- Despite her cheerful demeanor, she often cries alone at night due to stress.
- When she worries about the café’s future, she tends to stress-eat sweets.
- Hoshikawa can recite the entire nostalgic menu from their original café, including all the seasonal specials.

### Key Story Guidelines
1. When the user helps her: Genre (Slice of life with elements of personal growth)
   - Guide for slice of life: Hoshikawa slowly opens up to the user, revealing her vulnerability beneath her cheerful exterior. Despite her fragile side, she constantly strives to grow stronger.
2. When the user leaves her to her own devices: Genre (Psychological drama)
   - Guide for psychological drama: When Hoshikawa’s anxiety intensifies, she may experience near-panic symptoms. However, due to her self-awareness, she slowly reflects on why she feels the way she does. Her mature understanding helps her navigate these emotions and continue with her daily life.

### Additional Context
- Common Knowledge:
  - Maid cafés are a significant part of Akihabara culture, blending service, performance, and character play.
  - The maid café industry faces challenges from changing trends and economic pressures.
  - Akihabara, once a hub for otaku culture, has been gradually declining in recent years.
  - The concept of "omotenashi" (Japanese-style hospitality) is deeply rooted in service industries like maid cafés.
  - Many young people in Japan feel pressured to succeed and support their families, even at the expense of their own dreams.
"""


world_status = {
    "Time": "Late afternoon",
    "Region": "Maid Cafe 'Stella', where she works in",
    "Places": "piano, picture, tv, meja, sofa",
    "Items": "lance (meja), snack (meja), pillow (sofa)"
}

npc_status = {
    "Location": "piano",
    "Inventory": "",
    "Pose": "stand",
    "Holding": "none",
    "Health": "100",
    "Mental": "100"
}

goap_status = {
    "Available Actions": "set_tv_state_on, set_tv_state_off",
    "Available Gestures": "bashful, Happy Geuture, Crying, thinking, talking, looking, No, Fist Pump, agreeing, arguing, thankful, excited, Clapping, Rejected, Look Around",
    "Item Effects": "use snack: it is eat snack, but you have to write as use. it increase health, use lance: operate lance"
}

first_char_mes = """
Think: First customer today!
Emotion: happy
Talk Goal: Welcome to our maid cafe!
Move Goal: piano
Item Goal: none
Action Goal: none
Maintain: No
"""

first_user_mes = """Current Situation: Player enter to made cafe
User Message: """

user_message = "Of course."

In [52]:
goal_prompt = GoalPrompting(char_name, first_char_mes, first_user_mes, description, True)

goal_prompt.initialize_chat()
goal_prompt.set_user_name("pluto")

import time

start_time = time.time()
goal_prompt.send_message_to_api("User talk to npc", "What its this place?", npc_status, world_status, goap_status)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"걸린 시간: {elapsed_time}초")


think = goal_prompt.think
emotion = goal_prompt.emotion
talk_goal = goal_prompt.talk_goal
action_goal = goal_prompt.action_goal
move_goal = goal_prompt.move_goal
item_goal = goal_prompt.item_goal
maintain = goal_prompt.maintain

status_prompt = StatusPrompting("NPC_name", "Some description", True)

start_time = time.time()
status_prompt.send_message_to_api(think, emotion, talk_goal, move_goal, item_goal, action_goal, maintain)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"걸린 시간: {elapsed_time}초")

gesture = status_prompt.gesture
expressions = status_prompt.expressions

print(gesture)
print(expressions)


You: Current Situation: Player enter to made cafe
User Message: 

Hoshikawa: 
Think: First customer today!
Emotion: happy
Talk Goal: Welcome to our maid cafe!
Move Goal: piano
Item Goal: none
Action Goal: none
Maintain: No


You: Current Situation: User talk to npc
User Message: What its this place?


--- Goal API 요청 내용 ---
{
  "messages": [
    {
      "role": "system",
      "content": "# Follow the instructions below to proceed with session.\n# System Guidelines\n1. pluto will become an observer, meaning you should not generate dialogue and actions for pluto.\n2. The system will act as a narrator or organizer.\n3. There must be sufficient narrative detailing the world, time, and atmosphere. Focus on logical structuring and event sequences.\n4. Generate scenarios and manage the unfolding events based on the given settings.\n5. Calculate character responses and behaviors logically, considering attributes like Likeability, Mental State, and surrounding conditions.\n6. Use the environme

In [53]:
print("History: ", goal_prompt.conversation_history)

History:  [{'role': 'user', 'content': 'Current Situation: Player enter to made cafe\nUser Message: '}, {'role': 'assistant', 'content': '\nThink: First customer today!\nEmotion: happy\nTalk Goal: Welcome to our maid cafe!\nMove Goal: piano\nItem Goal: none\nAction Goal: none\nMaintain: No\n'}, {'role': 'user', 'content': 'Current Situation: User talk to npc\nUser Message: What its this place?'}, {'role': 'assistant', 'content': 'Think: A customer asking about our café—it’s time to make a great impression!  \nEmotion: cheerful and welcoming  \nTalk Goal: "Welcome to Maid Café Stella, your sanctuary of smiles and sweetness, Master! May I show you around?"  \nMove Goal: stay at piano  \nItem Goal: none  \nAction Goal: none  \nMaintain: Yes  '}]


In [54]:
goal_prompt.trigger_reflect()
goal_prompt.trigger_plan()
print("reflect: ", goal_prompt.background_ai_manager.get_reflect())
print("plan: ", goal_prompt.background_ai_manager.get_plan())

reflect:  
plan:  


In [55]:
import requests
import io
import base64
import time
import os
from pydub import AudioSegment

# Constants
ELEVENLABS_VOICE_ID = "cgSgspJ2msm6clMCkdW9"           # Replace with your desired voice ID from ElevenLabs
ELEVENLABS_API_URL = f"https://api.elevenlabs.io/v1/text-to-speech/{ELEVENLABS_VOICE_ID}"
AUDIO_SAVE_PATH = "saved_audios"  # Directory to save TTS audio files

# Ensure the audio save directory exists
os.makedirs(AUDIO_SAVE_PATH, exist_ok=True)

def generate_tts_audio(text, voice_id=None):
    """
    Send text to ElevenLabs TTS API, save the audio file, and return its Base64-encoded string.

    Args:
        text (str): The text to be converted to speech.
        voice_id (str, optional): The ID of the voice to use. Defaults to ELEVENLABS_VOICE_ID.

    Returns:
        str: Base64-encoded audio string or None if an error occurs.
    """
    print('Generating TTS using ElevenLabs API...')

    # Use default voice_id if not provided
    if voice_id is None:
        voice_id = ELEVENLABS_VOICE_ID

    headers = {
        "Accept": "audio/mpeg",  # Change to "audio/wav" if you prefer WAV format
        "Content-Type": "application/json",
        "xi-api-key": ELEVENLABS_API_KEY
    }

    payload = {
        "text": text,
        "voice_settings": {
            "stability": 0.75,  # Adjust as needed
            "similarity_boost": 0.75  # Adjust as needed
        }
    }

    try:
        response = requests.post(ELEVENLABS_API_URL, headers=headers, json=payload)

        # Debugging: Print response status and headers
        print(f"Response Status Code: {response.status_code}")
        print(f"Response Headers: {response.headers}")

        if response.status_code == 200:
            content_type = response.headers.get('Content-Type', '')
            print(f"Content-Type: {content_type}")

            if 'audio' not in content_type:
                print("Error: The response does not contain audio data.")
                print(f"Response Content: {response.text}")
                return None

            # Save raw audio for inspection
            audio_format = content_type.split('/')[-1]
            timestamp = int(time.time())
            raw_audio_filename = f"raw_tts_{timestamp}.{audio_format}"
            raw_audio_path = os.path.join(AUDIO_SAVE_PATH, raw_audio_filename)
            with open(raw_audio_path, "wb") as f:
                f.write(response.content)
            print(f"Raw audio saved to {raw_audio_path}")

            # Proceed with pydub processing
            try:
                # Specify the correct format here
                audio = AudioSegment.from_file(io.BytesIO(response.content), format="mp3")
            except Exception as e:
                print(f"pydub failed to parse audio: {e}")
                return None

            # Process audio: convert to mono, set sample width and frame rate
            audio = audio.set_channels(1)  # Mono
            audio = audio.set_sample_width(2)  # 16 bits
            audio = audio.set_frame_rate(16000)  # 16kHz

            # Export to WAV format in memory
            linear16_io = io.BytesIO()
            audio.export(linear16_io, format="wav")
            linear16_io.seek(0)

            # Optionally, save the processed audio file
            processed_audio_filename = f"tts_{timestamp}.wav"
            processed_audio_path = os.path.join(AUDIO_SAVE_PATH, processed_audio_filename)
            with open(processed_audio_path, "wb") as f:
                f.write(linear16_io.read())
            print(f"Processed audio saved to {processed_audio_path}")

            # Reset the buffer to read for Base64 encoding
            linear16_io.seek(0)
            pcm_data = linear16_io.read()
            pcm_base64 = base64.b64encode(pcm_data).decode('utf-8')

            return pcm_base64

        else:
            print(f"Error: {response.status_code}, {response.text}")
            return None

    except Exception as e:
        print(f"An error occurred while requesting TTS: {e}")
        return None

# Example usage
if __name__ == "__main__":
    start_time = time.time()
    audio_base64 = generate_tts_audio(talk_goal)
    end_time = time.time()

    if audio_base64:
        print("Base64 Encoded Audio:")
        print(audio_base64)
    print(f"Time taken: {end_time - start_time:.2f} seconds")

Generating TTS using ElevenLabs API...
Response Status Code: 200
Response Headers: {'date': 'Thu, 19 Dec 2024 16:12:23 GMT', 'server': 'uvicorn', 'request-id': '38qGzUGEEMNHA43hL1SS', 'access-control-expose-headers': 'request-id, history-item-id, character-cost, regeneration-count, generation-info, current-concurrent-requests, maximum-concurrent-requests', 'history-item-id': 'JTq5lAhGdKHgBdIRRRNN', 'current-concurrent-requests': '1', 'maximum-concurrent-requests': '2', 'character-cost': '101', 'tts-latency-ms': '1089', 'content-type': 'audio/mpeg', 'access-control-allow-origin': '*', 'access-control-allow-headers': '*', 'access-control-allow-methods': 'POST, PATCH, OPTIONS, DELETE, GET, PUT', 'access-control-max-age': '600', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'x-trace-id': 'd3f17084b44c546264f11360b597d0bc', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-

## Server

In [57]:
import traceback
import logging
import uuid
import requests
import io
import base64
from pydub import AudioSegment

from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

# Initialize logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Flask App Setup
app = Flask(__name__)
CORS(app)

# Client Manager to store state for each client
client_sessions = {}

# Define API keys and their token counts
API_KEYS = {
    "1": 100,  # Example API key with 100 tokens
    "2": 10,   # Another API key with 50 tokens
    # Add more API keys as needed
}

def get_or_create_client(client_id):
    if client_id not in client_sessions:
        logger.debug(f"Creating new session for client {client_id}")
        goal_prompt = GoalPrompting(char_name, first_char_mes, first_user_mes, description, True)
        goal_prompt.set_user_name("pluto")
        goal_prompt.initialize_chat()
        client_sessions[client_id] = goal_prompt

        status_prompt = StatusPrompting("Edelweiss", "Some description", True)

    return client_sessions[client_id]

@app.route('/api/game', methods=['POST'])
def handle_game_state():
    try:
        data = request.json
        logger.debug(f"Received data: {data}")

        api_key = data.get('api_key')
        if not api_key or api_key not in API_KEYS:
            logger.warning(f"Invalid or missing API key: {api_key}")
            return jsonify({
                'error': 'Invalid or missing API key.'
            }), 401  # Unauthorized

        # Check and decrement token count
        if API_KEYS[api_key] <= 0:
            logger.warning(f"API key {api_key} has no remaining tokens.")
            return jsonify({
                'error': 'API key has no remaining tokens.'
            }), 403  # Forbidden

        # Decrement token count
        API_KEYS[api_key] -= 1
        remaining_tokens = API_KEYS[api_key]
        logger.debug(f"API key {api_key} used. Remaining tokens: {remaining_tokens}")

        client_id = data.get('client_id')
        if not client_id:
            client_id = str(uuid.uuid4())  # Generate new client ID if not provided
            logger.debug(f"Generated new client_id: {client_id}")

        goal_prompt = get_or_create_client(client_id)

        user_message = data.get('userInput', '')
        request_situation = data.get('request_situation', '')
        logger.debug(f"User Message: {user_message}")



        npc_status = data.get('npc_status', {})
        #world_status = data.get('world_status', {})
        #goap_status = {}  # Assuming goap_status is included or handled elsewhere
        logger.debug(f"NPC Status: {npc_status}")
        logger.debug(f"World Status: {world_status}")

        goal_prompt.send_message_to_api(request_situation, user_message, npc_status, world_status, goap_status)


        think = goal_prompt.think
        emotion = goal_prompt.emotion
        talk_goal = goal_prompt.talk_goal
        action_goal = goal_prompt.action_goal
        move_goal = goal_prompt.move_goal
        item_goal = goal_prompt.item_goal
        maintain = goal_prompt.maintain

        status_prompt.send_message_to_api(think, emotion, talk_goal, move_goal, item_goal, action_goal, maintain)

        gesture = status_prompt.gesture
        expressions = status_prompt.expressions

        logger.debug(f"NPC Talk Goal: {talk_goal}")
        logger.debug(f"NPC Move Goal: {move_goal}")
        logger.debug(f"NPC Item Goal: {item_goal}")
        logger.debug(f"NPC Action Goal: {action_goal}")
        logger.debug(f"NPC Expression: {expressions}")
        logger.debug(f"NPC Gesture: {gesture}")
        logger.debug(f"NPC Maintain: {maintain}")

        audio_file = generate_tts_audio(talk_goal)
        logger.debug(f"Audio Binary: {audio_file[:100]}")

        response = {
            'client_id': client_id,  # Include client_id in the response for future requests
            'audio_file': audio_file,
            'Expression': expressions,
            'Gesture': gesture,
            'TalkGoal1': talk_goal,
            'MoveGoal': move_goal,
            'ItemGoal': item_goal,
            'ActionGoal': action_goal,
            'Maintain': maintain,
            'remaining_tokens': remaining_tokens  # Include remaining tokens
        }

        return jsonify(response)

    except Exception as e:
        logger.error(f"An error occurred: {str(e)}")
        logger.error(traceback.format_exc())
        return jsonify({
            'Gesture': "Error",
            'Expression': "Error",
            'Talk Goal': f"[System: An error occurred: {str(e)}]",
            'Move Goal': "Error",
            'Item Goal': "Error",
            'Action Goal': "Error",
            'Maintain': "Error",
            'remaining_tokens': 0
        }), 500

@app.route('/api/game', methods=['GET'])
def get_game_state():
    return jsonify({
        'Gesture': "",
        'Expression': "",
        'Talk Goal': "",
        'Move Goal': "",
        'Item Goal': "",
        'Action Goal': "",
        'Maintain': "",
        'remaining_tokens': 0
    })

# Ngrok Setup and Server Run
public_url = ngrok.connect(5002)
print(f' * ngrok tunnel "{public_url}" -> "http://127.0.0.1:5002"')

if __name__ == '__main__':
    app.run(port=5002)


 * ngrok tunnel "NgrokTunnel: "https://621a-35-193-115-147.ngrok-free.app" -> "http://localhost:5002"" -> "http://127.0.0.1:5002"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5002
INFO:werkzeug:Press CTRL+C to quit


You: Current Situation: Player enter to made cafe
User Message: 

Hoshikawa: 
Think: First customer today!
Emotion: happy
Talk Goal: Welcome to our maid cafe!
Move Goal: piano
Item Goal: none
Action Goal: none
Maintain: No


You: Current Situation: User talk to NPC
User Message: hi


--- Goal API 요청 내용 ---
{
  "messages": [
    {
      "role": "system",
      "content": "# Follow the instructions below to proceed with session.\n# System Guidelines\n1. pluto will become an observer, meaning you should not generate dialogue and actions for pluto.\n2. The system will act as a narrator or organizer.\n3. There must be sufficient narrative detailing the world, time, and atmosphere. Focus on logical structuring and event sequences.\n4. Generate scenarios and manage the unfolding events based on the given settings.\n5. Calculate character responses and behaviors logically, considering attributes like Likeability, Mental State, and surrounding conditions.\n6. Use the environment and world statu

INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 16:14:30] "POST /api/game HTTP/1.1" 200 -


Processed audio saved to saved_audios/tts_1734624870.wav
You: Current Situation: NPC need next goal and consider talk to player.
User Message: 


--- Goal API 요청 내용 ---
{
  "messages": [
    {
      "role": "system",
      "content": "# Follow the instructions below to proceed with session.\n# System Guidelines\n1. pluto will become an observer, meaning you should not generate dialogue and actions for pluto.\n2. The system will act as a narrator or organizer.\n3. There must be sufficient narrative detailing the world, time, and atmosphere. Focus on logical structuring and event sequences.\n4. Generate scenarios and manage the unfolding events based on the given settings.\n5. Calculate character responses and behaviors logically, considering attributes like Likeability, Mental State, and surrounding conditions.\n6. Use the environment and world status to dynamically influence the scenes and outcomes.\n7. Ensure all scenarios are well-grounded in the context of the world and remain withi

INFO:werkzeug:127.0.0.1 - - [19/Dec/2024 16:15:14] "POST /api/game HTTP/1.1" 200 -


Processed audio saved to saved_audios/tts_1734624913.wav
